**IMPORTANT NOTE**: Exercie 2 will only work on your own machine (not via noteable, I think). Also: You'll need to unzip the chrome driver in the folder where your notebook is.

# Exercise 1

Just using beautiful soup go to the description of this course and extract the number of SCQF level for it. http://www.drps.ed.ac.uk/22-23/dpt/cxcmse11427.htm

Spoiler alert: the content should be 'SCQF Level ...'.

*hint*: you can use in to check for occurance of a word in a sentence

In [59]:
if 'banana' in 'a sentence with bananas':
    print('this sentence has a word banana in it')

# if word in sentence:
#     print('this sentence has a word in it')

this sentence has a word banana in it


In [60]:
# !pip install bs4 
from urllib.request import urlopen
import os
from bs4 import BeautifulSoup

In [61]:
# website address
page = 'http://www.drps.ed.ac.uk/22-23/dpt/cxcmse11427.htm'

# open the url and store the website
website = urlopen(page)

# convert the website's content, for this a parser is needed. In this case a html parser
soup = BeautifulSoup(website, 'html.parser')

# Retrieve the cell that contains 'SCQF Level'
# Retrieve the cell that contains 'SCQF Credits'
table = soup.find('table', {'class':'sitstablegrid'})
for cell in table.findChildren('td'):
    if 'SCQF Level' in cell.text:
        print(cell.text)
        break

SCQF Level 11 (Postgraduate)


# Exercise 2

Again, we hope to find all reviews of Edinburgh castle, but now we don't have API key. 

**It is NOT suggested for any commerical use**, but for educational purposes, you can have a try!

We look at the Google map, Edinburgh castle: https://www.google.com/maps/place/Edinburgh+Castle/@55.9485977,-3.2021022,17z/data=!4m5!3m4!1s0x4887c79a2099c0f7:0x469a1eebe54c0a58!8m2!3d55.9485947!4d-3.1999135

And we try to find all reviews of Edinburgh castle, OR we can try to get all reviews.

In [30]:
# !pip install selenium 
import time
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [5]:
# define method that will create a browser, suitable to your operating system
import sys
def get_a_browser():
    if sys.platform.startswith('win32') or sys.platform.startswith('cygwin'):
        return webdriver.Chrome() # windows
    else:
        return webdriver.Chrome('./chromedriver') # mac

In [8]:
# This is Edinburgh castle
url = 'https://www.google.com/maps/place/Edinburgh+Castle/@55.9485947,-3.2021022,17z/data=!3m1!4b1!4m5!3m4!1s0x4887c79a2099c0f7:0x469a1eebe54c0a58!8m2!3d55.9485947!4d-3.1999135'

In [51]:
browser = get_a_browser()

# the url we want to open
# fill in the correct url for google map, Edinburgh castle!
# You can also try other places!

# the browser will start and load the webpage
browser.get(url)

# we wait 1 second to let the page load everything
time.sleep(1)

# Google map has cookies warning, just accept the cookies
# In the previous, we use a div, span, h1 to find the elements, however, if you need to find a specific element, 
# then xpath is a better option
xpath_for_accept_cookies = '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[2]/div/div/button'
accept_cookies = browser.find_element(By.XPATH, xpath_for_accept_cookies)
print(accept_cookies)
accept_cookies.click();
time.sleep(3)

# Get total number of reviews
xpath_for_total_number_of_reviews = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[2]/span/span'
total_number_of_reviews = browser.find_element(By.XPATH, xpath_for_total_number_of_reviews)
# The number we get is '(89,862) reviews', but we hope to get 89862!
total_number_of_reviews = int(total_number_of_reviews.text.split(" ")[0].replace('(', '').replace(')', '').replace(',', ''))
print('There are', total_number_of_reviews, 'reviews')

# go to all reviews page
xpath_for_all_reviews ='//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]'
browser.find_element(By.XPATH, xpath_for_all_reviews).click()
# some other xpath may work, you can have a try!
time.sleep(3)

print('Pleasae KEEP BROWSER OPEN!')

<selenium.webdriver.remote.webelement.WebElement (session="7caf545718cdbd47d31031a268687328", element="73A61B43A4B47CCB4754FB9D4515B3F1_element_20")>
There are 89863 reviews
Pleasae KEEP BROWSER OPEN!


In [ ]:
# Scroll down to access all reviews, first of all, let's find the element!

In [58]:
"""
There are many ways to scroll down
"""
### 1. Use xpath
xpath_for_scrollable_div = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]'
scrollable_div = browser.find_element(By.XPATH, xpath_for_scrollable_div)

### 2. use css selector
# css_for_scrollable_div = '#QA0Szd > div > div > div.w6VYqd > div:nth-child(2) > div > div.e07Vkf.kA9KIf'
# scrollable_div = browser.find_element(By.CSS_SELECTOR, css_for_scrollable_div)

#Scroll as many times as necessary to load all reviews
"""
Google map reviews display 10 per scoll down, so in total, we need to run round(total_number_of_reviews/10 - 1) times;
However Google has a limit maximum of requests 120.
"""
# for i in range(0,(round(total_number_of_reviews/10 - 1))):
for i in range(0,10):
    browser.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', 
            scrollable_div)
    time.sleep(1)
    
### 3. PAGE DOWN method
# Find the body of the page
# body = browser.find_element(By.TAG_NAME,'body')
# # Keep scrolling down using a simulation of the PAGE_DOWN button
# for _ in range(10):
#     body.send_keys(Keys.PAGE_DOWN)
#     time.sleep(1)

#-----------------------------------------------------------------
#extract the info
response = BeautifulSoup(browser.page_source, 'html.parser')
reviews = response.find_all('span', class_='wiI7pd', string=True)
dates = response.find_all('span', class_='rsqaWe', string=True)
rates = response.find_all('span', class_="kvMYJc")
users = response.find_all('div', class_='jftiEf fontBodyMedium')

reviews_text=[]
dates_text = []
users_text = []
rates_int = []

for i in range(len(reviews)):
    reviews_text.append(reviews[i].text)
    dates_text.append(dates[i].text)
    rates_int.append(int(rates[i]['aria-label'][0]))
    users_text.append(users[i]['aria-label'])

In [56]:
#Build up a pandas dataframe to store the dataset
df = pd.DataFrame({"users":users_text, 
                   "rates":rates_int, 
                   "review":reviews_text, 
                   "dates":dates_text})

In [57]:
df

,users,rates,review,dates
0,Malissa Scalco,5,Fantastic views and steeped in medieval histor...,a week ago
1,Claire Gudewich,5,Definitely book a guided tour with a person. O...,a week ago
2,Amber Heck,5,I recommend visiting the castle to any tourist...,a week ago
3,Rodrigo Sarai,5,Nice place to go with the whole family. The ti...,a week ago
4,Fred Leon,5,An awesome place to visit. The Access for disa...,2 weeks ago
5,Hock Sing Tan,5,We spent 3 hours in this Edinburgh iconic buil...,a week ago
6,Darren Mattice,4,The age of the castle is amazing and beautiful...,2 weeks ago
7,Tray Carlisle,5,It was definitely impressive to see firsthand....,a week ago
8,Fiorenza Fiora,5,Edinburgh Castle is a must-visit if you find y...,a week ago
9,Sukhmani Singh,5,Great place to dig deep into the Scottish hist...,a month ago


#  Exercise 3: Get Property info from ESPC

About ESPC:

ESPC is the home of property, and the first-choice property portal for home buyers and sellers in Edinburgh, the Lothians, Fife and the Borders.

* Find recent property listings
* Write the necessary info to a csv file

In [62]:
# Step 1: Import necessary libraries

In [65]:
import time
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

## Here are the code to scrape the property information, please fill the blank

In [66]:
# Define the EspcScraper class
class EspcScraper:
    # Initializer or constructor method
    def __init__(self):
        # Initialize a list to store the results
        self.results = []
    
    # Method to fetch the HTML content of a given URL
    def fetch(self, url):
        print('HTTP GET request to URL: %s' % url, end='')
        res = requests.get(url)  # Perform the HTTP GET request
        print(' | Status code: %s' % res.status_code)
        return res

    # Method to parse the HTML content and extract relevant data
    def parse(self, html):
        # Parse the HTML content using BeautifulSoup
        content = BeautifulSoup(html, 'lxml')
        # Find all div elements with the 'infoWrap' class
        # This is because the web page is structured in such a way that, card by card
        cards = content.findAll('div', {'class': 'infoWrap'})
        
        # Define delimiters for splitting the title
        delimiters = ":|,"
        
        # Iterate over each card and extract data
        for card in cards:
            # # Extract the hyperlink
            link = 'https://espc.com' + card.find('a')['href']
            
            # title includes property type, address, town and postcode
            title = card.find('h3', {'class': 'propertyTitle'}).text
            title_part = re.split(delimiters, title)
            postcode = title_part[-1].strip()
            if postcode.count(' ') > 1:
                town = postcode.split()[0]
                postcode = ' '.join(postcode.split()[-2:])
            else:
                town = title_part[2].strip()

            # description
            description = card.find('div', {'class': 'description'})
            
            # price & offer
            offer_type = card.find('span', {'class': 'offersOver'}).text
            price = card.find('span', {'class': 'price'}).text
            
            # facilities
            facilities = card.find('div', {'class': 'facilities'}).text
            # Ensure the list is at least 3 elements long by extending with 'U'
            facilities = (facilities + 'U' * 3)[:3]

            try:
                agent = card.find('div', {'class': 'logoWrap'}).find('img')['alt']
            except:
                agent = 'N/A'
            
            # Append the extracted information to the results list
            self.results.append({
                'offer_type': offer_type[:-len(price) - 1],
                'price': price.strip(),
                'property_type': title_part[0].strip(),
                'address': title_part[1].strip(),
                'town': town,
                'postcode': postcode,
                'area': postcode.split()[0],
                'beds': facilities[0],
                'toilets': facilities[1],
                'living_rooms': facilities[2],
                'description': description.text.split('\n', 1)[0],
                'link': link,
                'parking': 'parking' in description,
                'allocated': 'allocated' in description,
                'agent': agent
            })

    def run(self,url, result_saving=False):
        res = self.fetch(url)
        self.parse(res.text)
        
        # get number of pages
        li_tags = BeautifulSoup(res.text, 'lxml').select('ul.paginationList > li')
        n_pages = int(li_tags[-2].text)
        
        # get all pages
        insert_position = url.find('?') + 1  # +1 to insert after the '?'
        for page in range(2, n_pages + 1):
            string_to_insert = 'p=' + str(page) + '&'
            new_url = url[:insert_position] + string_to_insert + url[insert_position:]
            new_res = self.fetch(new_url)
            self.parse(new_res.text)
            time.sleep(2)
            
        df = pd.DataFrame(self.results)
        # Just clean the empty rows
        df.drop_duplicates(subset=['agent',
                                   'address',
                                   'price'],
                           inplace=True)
        
        # If result_saving is True, then save results
        if result_saving:
            print('Saving ', len(df), ' items to csv file (espc.csv)...')
            df.to_csv('espc.csv', index=False,
                      encoding='utf-8-sig')
        
        return df

In [67]:
# For example, 
url = 'https://espc.com/properties?locations=edinburgh&minbeds=2plus&maxprice=300000'
scraper = EspcScraper()
scraper.run(url, result_saving=True)

HTTP GET request to URL: https://espc.com/properties?locations=edinburgh&minbeds=2plus&maxprice=300000 | Status code: 200
HTTP GET request to URL: https://espc.com/properties?p=2&locations=edinburgh&minbeds=2plus&maxprice=300000 | Status code: 200
HTTP GET request to URL: https://espc.com/properties?p=3&locations=edinburgh&minbeds=2plus&maxprice=300000 | Status code: 200
HTTP GET request to URL: https://espc.com/properties?p=4&locations=edinburgh&minbeds=2plus&maxprice=300000 | Status code: 200
HTTP GET request to URL: https://espc.com/properties?p=5&locations=edinburgh&minbeds=2plus&maxprice=300000 | Status code: 200
HTTP GET request to URL: https://espc.com/properties?p=6&locations=edinburgh&minbeds=2plus&maxprice=300000 | Status code: 200
HTTP GET request to URL: https://espc.com/properties?p=7&locations=edinburgh&minbeds=2plus&maxprice=300000 | Status code: 200
HTTP GET request to URL: https://espc.com/properties?p=8&locations=edinburgh&minbeds=2plus&maxprice=300000 | Status code: 

,offer_type,price,property_type,address,town,postcode,area,beds,toilets,living_rooms,description,link,parking,allocated,agent
0,Fixed Price,"£195,000",End Terraced House,15 Whitehill Street,Newcraighall,EH21 8RB,EH21,2,1,2,McEwan Fraser Legal is delighted to present th...,https://espc.com/property/15-whitehill-street-...,False,False,McEwan Fraser Legal - Edinburgh
1,Offers Over,"£125,000",End Terraced House,38 Bogwood Road,Mayfield,EH22 5DZ,EH22,2,1,1,McEwan Fraser Legal is delighted to present th...,https://espc.com/property/38-bogwood-road-mayf...,False,False,McEwan Fraser Legal - Edinburgh
2,Offers Over,"£280,000",Second Floor Flat,128 (flat 9),McDonald Road,EH7 4QU,EH7,2,2,1,Well appointed two bedroom second floor apartm...,https://espc.com/property/128-flat-9-mcdonald-...,False,False,Connor Malcolm - PROPERTY DEPT
3,Offers Over,"£235,000",Ground Floor Flat,163/3 Easter Road,Edinburgh,EH7 5QB,EH7,2,2,1,This attractive and generously proportioned gr...,https://espc.com/property/163-3-easter-road-ed...,False,False,Neilsons - Property Centre
4,Offers Over,"£230,000",Triplex Apartment,2/1 Colonsay Way,Granton,EH5 1FB,EH5,2,2,1,Set in the heart of a modern development surro...,https://espc.com/property/2-1-colonsay-way-gra...,False,False,Warners Solicitors - Property Department
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,Fixed Price,"£285,000",Terraced House,2 Lochiel Gardens,,EH17 8WB,EH17,3,1,1,Part exchange available! McEwan Fraser is deli...,https://espc.com/property/2-lochiel-gardens-ed...,False,False,McEwan Fraser Legal - Edinburgh
388,Offers Over,"£185,000",First Floor Flat,17/2 St. Triduanas Rest,EDINBURGH,EH7 6NF,EH7,2,1,1,This modern first floor flat is situated withi...,https://espc.com/property/17-2-st-triduanas-re...,False,False,Beveridge & Kellas - LEITH WALK
389,Fixed Price,"£275,000",Flat,47/2 Blackfriars Street,,EH1 1NB,EH1,2,1,1,McEwan Fraser is delighted to present this mag...,https://espc.com/property/47-2-blackfriars-str...,False,False,N/A
390,Fixed Price,"£220,000",Second Floor Flat,1/6 Oswald Terrace,Corstorphine\r\nEDINBURGH,EH12 7TS,EH12,2,1,1,Fantastic opportunity to acquire a charming se...,https://espc.com/property/1-6-oswald-terrace-c...,False,False,TC Young


You can also try to get all other properties you're interested in.